Import classes and functions from twoBodyPINN.py and odeintegrator.py

Classes are NeuralNet and State. Functions are showProgress, showProgress2, trainNetwork, and showTrainingResult

In [1]:
from twoBodyPINN import *
#import seaborn as sns
#sns.set()
%matplotlib qt5

Running on CPU



1) Circular orbit (eccentricity $=0$)

In [2]:
#Using keyword arguments
an,nn,totalLoss,s,t_plot = showProgress(Tmax=4*np.pi,timesteps=61,initial=[1.,0,0,1],
                                        depth=2,width=32,lr=1,weights=[10,1,10,0.5,0])

In [3]:
#show training in real space
an,nn,totalLoss,s,t_plot = showProgress2(Tmax=4*np.pi,timesteps=61,initial=[1.,0,0,1],
                                        depth=2,width=16,lr=1,weights=[10,1,10,0.1,0])

In [4]:
#Using dictionary input
args = {"Tmax":4*np.pi,
     "timesteps":61,
     "initial":[1.,0.,0,1],
     "depth":2,
     "width":32,
     "lr":1,
     "weights":[10,1,10,0.01,0]}

an,nn,totalLoss,s,t_plot = showProgress(**args)

In [3]:
#show training progress
showTrainingResult(nn,t_plot,totalLoss,s)

2) Eccentric elliptical orbit (eccentricity $\approx 0.5$)

In [11]:
args = {"Tmax":8*np.pi,
     "timesteps":102,
     "initial":[1.,0.,0,1.2],
     "depth":2,
     "width":32,
     "lr":0.5,
     "weights":[10,1,10,0.1,0]}

an,nn,totalLoss,s,t_plot = showProgress2(**args)

In [12]:
showTrainingResult(nn,t_plot,totalLoss,s)

3) Eccentric elliptical orbit (eccentricity $\approx 0.82$) 

still need to find good parameters

In [19]:
args = {"Tmax":32*np.pi,
     "timesteps":281,
     "initial":[1.,0.,0,1.35],
     "depth":4,
     "width":62,
     "lr":0.125,
     "weights":[30,1,5,0.1,0]}

an,nn,totalLoss,s,t_plot = showProgress(**args)

In [17]:
showTrainingResult(nn,t_plot,totalLoss,s)

Without showing real time progress

In [2]:
args = {"Tmax":6*np.pi,
     "timesteps":514,
     "initial":[2.,0.5,0,0.5],
     "depth":10,
     "width":43,
     "lr":0.23,
     "weights":[10.27,0.54,10.90,0.00045,0],
     "epochs":10000}

nn,totalLoss,s,t_plot,flag,validationLoss = trainNetwork(**args)
showTrainingResult(nn,t_plot,totalLoss,s,validationLoss)

Radial motion

In [88]:
args = {"Tmax":6*np.pi,
     "timesteps":102,
     "initial":[1.,1.25,0,0],
     "depth":2,
     "width":32,
     "lr":0.5,
     "weights":[10,1,10,0.,0],
     "epochs":2000}

nn,totalLoss,s,t_plot,flag,validationLoss = trainNetwork(**args)
showTrainingResult(nn,t_plot,totalLoss,s,validationLoss)

Hyperbolic orbit

In [81]:
args = {"Tmax":1*np.pi,
     "timesteps":101,
     "initial":[-2.,1,-1,1],
     "depth":2,
     "width":32,
     "lr":0.5,
     "weights":[10,1,10,0.,0],
     "epochs":2000}

nn,totalLoss,s,t_plot,flag,validationLoss = trainNetwork(**args)
showTrainingResult(nn,t_plot,totalLoss,s,validationLoss)

In [15]:
from scipy.optimize import curve_fit

sol = nn(t_plot).detach().numpy()
x,px,y,py = sol[:,0],sol[:,1],sol[:,2],sol[:,3]
r = np.sqrt(x**2+y**2)
phi = np.arctan2(y,x)
L = x[0]*py[0]-y[0]*px[0]
E = 0.5*(px[0]**2+py[0]**2)-1/np.sqrt(x[0]**2+y[0]**2)
p,e = L**2,np.sqrt(1+2*E*L**2)

def rfunc(phi,alpha):
    return p/(1+e*np.cos(phi-alpha))

[alpha],_ = curve_fit(rfunc,phi,r,p0=[0])
print(p,e,alpha)

relErr = np.abs(rfunc(phi,alpha)/r-1)
plt.plot(relErr)
print(np.mean(relErr))

plt.plot(rfunc(phi,alpha))
plt.plot(r)

1.018719203650619 0.00810745513485646 -0.9328714579903249
0.045711502


In [42]:
#elliptic
args = {"Tmax":8*np.pi,
        "initial":[1,0,0,1.25],
        "timesteps":514,
        "depth":10,
        "width":43,
        "lr":0.23,
        "weights":[10.27,0.54,10.90,0.00045],
        "epochs":10000,
        "seed":2745
}

nn,totalLoss,s,t_plot,flag,validationLoss = trainNetwork(**args)
showTrainingResult(nn,t_plot,totalLoss,s,validationLoss)

In [48]:
args = {"Tmax":2*np.pi,
        "initial":[-1,1.1,-1,1],
        "timesteps":514,
        "depth":2,
        "width":16,
        "lr":0.23,
        "weights":[10.27,0.54,10.90,0.00045],
        "seed":2745
}
an,nn,totalLoss,s,t_plot = showProgress2(**args)

In [43]:
showTrainingResult(nn,t_plot,totalLoss,s,validationLoss)

In [5]:
import seaborn
seaborn.set()

In [83]:
#hyperbolic
args = {"Tmax":np.pi,#1.3,
        "initial":[-2.,1,-1,1],#[-1,1.48,-1,1],
        "timesteps":514,
        "depth":10,
        "width":43,
        "lr":0.23,
        "weights":[10.27,0.54,10.90,0.000],
        "epochs":10000,
        "seed":2745
}

nn,totalLoss,s,t_plot,flag,validationLoss = trainNetwork(**args)
showTrainingResult(nn,t_plot,totalLoss,s,validationLoss)

In [84]:
showTrainingResult(nn,t_plot,totalLoss,s,validationLoss)

In [86]:
s = State([0.71,0.71],[0.88,0.88])
s.evolve(0.01,)
plt.plot(s.x,s.y)

In [108]:
#singular / radial
args = {"Tmax":6.5*np.pi,
        "initial":[0.87,1.08,0.5,0.625],
        "timesteps":514,
        "depth":10,
        "width":43,
        "lr":0.23,
        "weights":[10.27,0.54,10.90,0.000],
        "epochs":10000,
        "seed":2745
}

nn,totalLoss,s,t_plot,flag,validationLoss = trainNetwork(**args)
showTrainingResult(nn,t_plot,totalLoss,s,validationLoss)

In [91]:
showTrainingResult(nn,t_plot,totalLoss,s,validationLoss)

In [107]:
args = {"Tmax":6.5*np.pi,
        "initial":[0.87,1.08,0.5,0.625],#[1,1.25,0,0],
        "timesteps":514,
        "depth":10,
        "width":43,
        "lr":0.23,
        "weights":[10.27,0.54,10.90,0.000],
        "seed":2745
}
_ = showProgress(**args)